**Data Sources**

In my analysis, I will be considering Singapore, Jakarta, Mumbai & Bangalore as key locations because of major business presence & thus driving cross-continent food habits. I will consider forsquare APIs to search key venues, trending ones, & specific italian restaurant. Some of API syntax as follows:

Extract all key venues in a radius parameter of 3000:
https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={}&v{}&limit=3000&radius=3000'

Extract trending venues, to check if our query for Italian food is one of them:
https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={}&v={}&limit=3000&radius=3000'

Extract all Italian venues:
https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={}&v={}&query=Italian&limit=3000&radius=3000'

Extract venue rating, tips related info:
https://api.foursquare.com/v2/venues/venue_id?{}?client_id={}&client_secret={}&v={}

All APIs will be used for 4 locations, & analysed using folium to check density, & then, reccomend two locations to XYZ.